###### The cell below loads the visual style of the notebook when run.

In [0]:
from IPython.core.display import HTML
css_file = '../../styles/styles.css'
HTML(open(css_file, "r").read())

# Fitting an isochrone to your data

<section class="objectives panel panel-warning">
<div class="panel-heading">
<h2><span class="fa fa-certificate"></span>Learning Objectives</h2>
</div>
</section>

> * Understand the location of co-eval stars in the HR diagram, and how it changes with age.
> * Learn how to use the ```isochrones``` library to create a model of co-eval stars.
> * Compare stellar models to your own open cluster data to find the best-fitting age and errors.


## Theory

We work on the assumption that all the stars in a cluster are born at the same time (are *co-eval*), and have the same metallicity. Therefore, the main factor that accounts for the different properties of the individual stars is that they have different masses. 

Having produced a colour-magnitude diagram of our open cluster in previous lab sessions, in this session we compare to stellar models in order to calculate the age, distance and [interstellar extinction](http://slittlefair.staff.shef.ac.uk/teaching/phy241/lectures/L03/index.html) of the open cluster.

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Read in your data</h2>
</div>
</section>

> At the end of the previous session, you should have created a CSV file containing apparent $V$ magnitudes, and $B-V$ colours for all of the stars in your open cluster. 

> Upload that file to somewhere on CoCalc (probably easiest to load into the same folder as this notebook). If you haven't finished the previous session yet, use the example CSV file
> included in this assignment ```example_data.csv```, which has only two columns ($V$ and $B-V$)

> Write some code that reads in the CSV file into two arrays - one for the $V$ magnitudes and one for the $B-V$ colours.

In [0]:
# WRITE YOUR CODE HERE

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Plot your colour-magnitude diagram</h2>
</div>
</section>

> Write a function that takes these two arrays and plots a colour-magnitude diagram ($V$ magnitude on the y-axis, $B-V$ on the x-axis). You will want to invert the y-axis so that brighter stars are at the top. You can use the matplotlib function ```axis.invert_yaxis()``` to do this.

> For use later on, your function will want to **return the matplotlib axis** that is created inside the function.

> Use your function to plot the colour-magnitude diagram (CMD).

In [0]:
# YOUR CODE HERE

## The `isochrones` library

The `isochrones` library is not built into Python, but it is installed on the Python3 (System-Wide) kernel on CoCalc. If you are running on your own laptop, you will have to install it yourself (`pip install isochrones`). The `isochrones` library allows you to compute the location in a colour-magnitude diagram of a co-eval population of stars, based on pre-computed grids of evolutionary stellar models.

You can use several sets of stellar models with the library, but we will use the so-called "MIST" stellar models, detailed in [Choi et al 2016](https://ui.adsabs.harvard.edu/abs/2016ApJ...823..102C/abstract).

You can import the MIST Isochrones using the following code:

In [0]:
from isochrones.mist import MIST_Isochrone

Once we've imported the library, you can create an isochrone object by specifying the filters you are interested in, like so:

In [0]:
iso = MIST_Isochrone(bands=['B','V'])

Here we've stored an *isochrone object* in the variable named  `iso`. This object contains a single function `isochrone`, that allows us to calculate the CMD location of stars of a given age. The single argument of this function is $\log_{10}$ of the age in years, so, for a 1.7 Gyr cluster we would use:

In [0]:
model = iso.isochrone(9.235)  # note the argument is log10(1.7e9)
print(type(model))

The `isochrone` function returns a `DataFrame` object from the `pandas` library. We haven't time or space to go into the `pandas` library in this course, except to say that many people love it for dealing with  data tables. For now, all we need to know is we can access the theoretical $B$ and $V$ magnitudes like so:

In [0]:
model_b = model.B_mag
model_v = model.V_mag
# calculate B-V for this model
model_bv = model_b - model_v

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Plot the isochrone</h2>
</div>
</section>

> Using the isochrones library, and your function from earlier, plot an isochrone on your HR diagram. Use an isochrone of approximately the correct age for your cluster.


In [0]:
# WRITE YOUR CODE HERE

# Distance, extinction and reddening

If you've done the last step correctly, you should find the isochrone lies nowhere near your data. What's going on?

The answer is that your data are in **apparent** magnitudes, **and** the light from the stars has been extincted and reddened by dust lying between us and the cluster. The `isochrones` library returns **absolute**, unreddened magnitudes.

The conversion between absolute and apparent magnitudes depends upon the distance to the cluster:

$$m - M = 5 \log_{10} (d/10),$$

where $d$ is the distance in parsecs. Notice that this will affect the magnitudes, but not the colour! This means the effect of distance is to move the isochrone vertically on the CMD.

What about reddening and extinction? Just like extinction by dust in our own atmosphere, interstellar extinction will make the stars fainter, and redder. So, whilst a finite distance moves an star vertically in the CMD, dust along the line of sight will move a star down and to the right.

We can write that the total extinction in the $V$-band, in magnitudes, is $A_V$. This means that if the true magnitude of a star is $V_0$, the observed magnitude is $V = V_0 + A_V$. However, the dust also makes the star redder, so we can write that the true colour of the star is related to the observed colour is $(B-V) = (B-V)_0 + E(B-V)$. The quantity $E(B-V)$ is known as the colour excess, or reddening.

Since both $A_V$ and $E(B-V)$ are related to the amount of dust between us and the cluster, it should not surprise you that they are related. It turns out that $A_V = 3.1 E(B-V)$; see [Shultz & Wiemer (1975)](https://ui.adsabs.harvard.edu/abs/1975A%26A....43..133S/abstract), for example.

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Apply distance and reddening</h2>
</div>
</section>

> Write a new function that accepts four arguments: a matplotlib axis, the logarithm of the age, the distance and the extinction, and plots an isochrone on the provided axis. Correct the V magnitudes and the B-V colours of the isochrone for distance and reddening. I've provided a template to get you started.

> Use your function in combination with others you've plotted above to plot your CMD with three isochrones on top of it. Plot one isochrone of approximately the correct age, distance and reddening. Then plot two more; one a Gyr older, the other a Gyr younger.

In [0]:
def plot_isochrone(axis, log_age, distance, a_v):
    """
    Plots a MIST group isochrone on the axis supplied by the user

    Parameters
    ----------
    axis: matplotlib.axis
        the matplotlib axis object on which we want to plot
    log_age: float
        logarithm of the age in years (base 10)
    distance: float
        distance to cluster in parsecs
    a_v: float
        V-band extinction to cluster, in magnitudes
    """
    # YOUR CODE HERE

In [0]:
# USE YOUR FUNCTION HERE

---------
<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h1>Homework #7</h1>
<h2><span class="fa fa-pencil"></span>  Find the best fitting isochrone</h2>
</div>
</section>

> Now it's time to use the code you wrote above to fit the isochrones to your data and find the best fitting model. Follow the instructions below to find the age, distance and reddening to your cluster, with errors.

By trial and error, using the functions written above, you can alter the distance, reddening and age until the isochrone sequence lines up with the data.

However, if you only fit the position of the main-sequence stars, you could do this for an isochrone of any age! Even if the isochrone is the wrong age there'll be a value of the reddening and distance which will make the main sequence of the isochrone line up with the main sequence in the data. How then do we find the age of the cluster?

The answer is that only an isochrone of the correct age will simultaneously fit the main sequence and the location of the turn-off from the main sequence. Thus you can find the age, distance and reddening all from one colour-magnitude diagram by trying isochrones of different ages until you find an acceptable fit!

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-question"></span>  Q1: Find the best fitting isochrone (4 points)</h2>
</div>
</section>

> Using this method, find the distance, reddening and age that best fits your cluster. Make a plot of the best fitting age.

In [0]:
# YOUR CODE HERE

## Uncertainties

How to judge our uncertainties in the three quantities? One way is to try out isochrones of different ages. Suppose we try an isochrone which is 200 Myr older than our best fit above. We'd need to change the distance and extinction again to get it as close as possible to the data, but once we'd done that, it might still be a "reasonable" fit; i.e it looks OK by-eye. 

What if we tried one 400 Myr older? Perhaps, even after tweaking the distance and reddening, we still had a poor fit to the data, again judged by-eye. In this case, we'd conclude that a cluster 200 Myr older was consistent with our data, but one 400 Myr older is not. Therefore, an uncertainty of 200 Myr on the age would seem reasonable.

We could use the spreads in distance and extinction we found to estimate uncertainties in those properties, too.

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-question"></span>  Q2: Finding our errors (6 points)</h2>
</div>
</section>

> Use the approach above to find error bars for your age, distance and extinction to the cluster. Make a plot that has your best fitting isochrone on it, along with the isochrones that represent the worst fit you think is still OK. Clearly label which is which.

In [0]:
# YOUR CODE HERE.

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<h2><span class="fa fa-question"></span> Q3: Systematic Errors (2 points)</h2>
</div>
</section>

> The technique above will give you a good idea of your statistical errors; i.e how the scatter in the CMD affects your ability to accurately judge the age. However, there may be sources of *systematic* errors; that might pull all the stars around in the CMD and systematically affect the age, distance or extinction you derive. In the markdown cell below, briefly outline what these may be, and what effect they would have.

**Write your answer here**